In [2]:
import lcdb
import numpy as np
from sklearn.linear_model import LinearRegression
import sklearn.model_selection
from sklearn import metrics
import pandas as pd
import scipy
from tqdm import tqdm
from functools import reduce
import random
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import concurrent.futures


outer_seed = 0
inner_seed_index = 0
num_seeds = 5
outer_seeds = list(range(0,5))
inner_seeds = list(range(num_seeds * inner_seed_index, num_seeds * (inner_seed_index + 1)))

In [ ]:
def calc_slopes_by_train(anchors, openml_id, learner_name, outer_seeds, inner_seeds, plotting=False):
    X, y = lcdb.get_dataset(openml_id)
    slopes = []
    n = len(anchors)
    
    # Code from LCDB paper
    learner_params = {}
    if learner_name == "SVC_linear":
        learner_name = "sklearn.svm.LinearSVC"
    elif learner_name == "SVC_poly":
        learner_name = "sklearn.svm.SVC"
        learner_params = {"kernel": "poly"}
    elif learner_name == "SVC_rbf":
        learner_name = "sklearn.svm.SVC"
        learner_params = {"kernel": "rbf"}
    elif learner_name == "SVC_sigmoid":
        learner_name = "sklearn.svm.SVC"
        learner_params = {"kernel": "sigmoid"}
    
    
    for idx,anchor in tqdm(enumerate(anchors)):
        points = None
        if idx == 0:
            points = range(anchor, anchor+21)
        if idx == n - 1:
            points = range(anchor-20, anchor+1)
        else:
            points = range(anchor-10, anchor+11)
        points = np.array(points).reshape(-1,1)
        err_lin_reg = []
        err_lin_points = []
        slp = []
        for outer_seed in outer_seeds:
            for inner_seed in inner_seeds:
                measurements = []
                for anch in points:
                    info = lcdb.get_entry(learner_name, {}, X,y, anch[0], outer_seed, inner_seed)
                    y_pred = encoder.decode_label_vector(info['y_hat_train'])
                    y_true = encoder.decode_label_vector(info['y_train'])
                    err = 1 - metrics.accuracy_score(y_true, y_pred)
                    measurements.append(err)
                    err_lin_points.append(anch)
                lin_reg = LinearRegression()
                lin_reg.fit(points, measurements)
                slp.append(lin_reg.coef_[0])
        slopes.append(np.mean(slp))

    return slopes

@ignore_warnings(category=ConvergenceWarning)
@ignore_warnings(category=UserWarning)
@ignore_warnings(category=RuntimeWarning)
def calc_monotonicity_one_lc(openml_id, learner_name):
    curve = lcdb.get_curve(openml_id, learner_name)
    errors = [[1-y for y in x] for x in curve[1]] # Transform accuracy in error rate
    slopes_curr = calc_slopes_by_train(curve[0], dbid, lern, outer_seeds, inner_seeds)
    
    for j in range(0, len(slopes_curr) - 1):
        if slopes_curr[j] > 0 and slopes_curr[j+1] > 0:
            return slopes_curr, True
    
    if slopes_curr[(len(slopes_curr) - 1)]:
        return slopes_curr, True
    
    return slopes_curr, False